Task:   Detection of owls and sheep using pretrained YOLOv8 on OpenImages-v7 dataset

Author: Josipa 

INSTALLATION AND IMPORTATION OF THE REQUIRED LIBRARIES

In [1]:
# Installation
%pip install fiftyone
%pip install ultralytics
%pip install polars
%pip install torch

In [1]:
# Importation
import os
import fiftyone as fo
from   ultralytics import YOLO
import torch
import random
import shutil

DOWNLOADING DATASET FOR MODEL DEPLOYMENT

In [3]:
# Downloading dataset from Google apis storages - OpenImages-v7, using Fiftyone library
'''
data = fo.zoo.load_zoo_dataset("open-images-v7",
                                label_types = ["detections"],    # Specifying that the task is detection to ensure the corresponding labels are downloaded.
                                classes     = ["Owl", "Sheep"])  # Downloading only images with owls and sheep
'''

# Exporting the data in a suitable format for YOLOv8
'''
data.export(export_dir   = "./export_data/",       # Choosing the directory where the data will be exported
            dataset_type = "fiftyone.types.YOLOv5Dataset")       # Choosing an extraction format for the data
'''

'\ndata.export(export_dir   = "/home/josipa/Documents/data/",\n            dataset_type = "fiftyone.types.YOLOv5Dataset")\n'

DIVISION OF THE ORIGINAL DATASET INTO THREE REQUIRED PARTS: TRAIN, VALIDATION, AND TEST

In [2]:
# The whole dataset has been downloaded into one folder so it must be manually divided into a three sets
# The decision is to divide it according to its size in the ratio 8.5 : 1 : 0.5
'''
train_set_size      = round((17/20) * 2909)  # Size of a train set
validation_set_size = round((2/20) * 2909)   # Size of a validation set
test_set_size       = round((1/20) * 2909)   # Size of a test set

txt_names  = [os.path.splitext(filename)[0] for filename in os.listdir('./export_data/')] # Txt files names connected to a images 

# Dividing txt names randomly into three lists

random.shuffle(txt_names)  # Shuffling the original list

txt_names_train = txt_names[:train_set_size]                                      # First  - train part
txt_names_valid = txt_names[train_set_size:train_set_size + validation_set_size]  # Second - validation part
txt_names_test  = txt_names[train_set_size + validation_set_size:]                # Third  - test part

imgs_train_names   = [name + ".jpg" for name in txt_names_train] # Names of image files that belong in the training set
labels_train_names = [name + ".txt" for name in txt_names_train] # Names of txt files that belong in the training set

imgs_valid_names   = [name + ".jpg" for name in txt_names_valid] # Names of image files that belong in the validation set
labels_valid_names = [name + ".txt" for name in txt_names_valid] # Names of txt files that belong in the validation set

imgs_test_names    = [name + ".jpg" for name in txt_names_test]  # Names of image files that belong in the test set
labels_test_names  = [name + ".txt" for name in txt_names_test]  # Names of txt files that belong in the test set

# Creating three new folders with names train, valid and test, and creating two folder inside of them called images and labels
new_folders_names = ['train', 'valid', 'test']   

destination_folder_imgs   = []
destination_folder_labels = []

for i in range(len(new_folders_names)):
    destination_folder = f"./export_data/{new_folders_names[i]}/images"
    os.makedirs(destination_folder, exist_ok = True) # Creating the folder if it doesn't exist
    destination_folder_imgs.append(destination_folder)

    destination_folder = f"./export_data/{new_folders_names[i]}/labels"
    os.makedirs(destination_folder, exist_ok = True) # Creating the folder if it doesn't exist 
    destination_folder_labels.append(destination_folder)

source_folder_imgs   = "./export_data/images" # Source folder where the images are located
source_folder_labels = "./export_data/labels" # Source folder where the labels are located

imgs_names    = [[imgs_train_names],   [imgs_valid_names],   [imgs_test_names]] # Creating a nested list with lists that contain of images names for different sets

for i in range(len(imgs_names)):         # Loop through all items in imgs_names
    for img_name in imgs_names[i][0]:    # Loop through elements in imgs_names[i][0]
    
        source_path_imgs = os.path.join(source_folder_imgs, img_name)               # Defining a path to an image that will be copied
        destination_path_imgs = os.path.join(destination_folder_imgs[i], img_name)  # Defining a path where an image will be copied
        shutil.copy(source_path_imgs, destination_path_imgs)                        # Copying an image

        new_txt_name = f"{img_name[:-4]}.txt"
        source_path_labels = os.path.join(source_folder_labels, new_txt_name)              # Defining a path to the txt that will be copied
        destination_path_labels = os.path.join(destination_folder_labels[i], new_txt_name) # Defining a path where the txt will be copied
        shutil.copy(source_path_labels, destination_path_labels)                           # Copying an txt file

'''

DEFINITION OF A PRETRAINED MODEL

In [5]:
# Downloading pretrained model from the Ultralytics library and saving it in the variable "model"
model  = YOLO('yolov8m.pt')

# Transferring model to a desired device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #  Desired device is GPU, if it is not available then using CPU
model  = model.to(device)

100%|██████████| 49.7M/49.7M [00:04<00:00, 10.9MB/s]


MODEL TRAINING

In [ ]:
# Model training with adjusted hyperparameters
model.train(data         = './data/data.yaml',  # Path to a .yaml file
            epochs       = 100,                                            # Number that represents how many of full passess will be over the entire data
            patience     = 8,                                              # Number of epochs to wait without improvement
            batch        = 8,                                              # Batch size
            imgsz        = 720,                                            # Target image size for training
            save         = True,                                           # Enabling saving of checkpoints and final model weights
            workers      = 8,                                              # Number of worker threads for data loading
            lr0          = 0.01,                                           # Initial learning rate
            lrf          = 0.01,                                           # Final learning rate
            momentum     = 0.937,                                          # Momentum factor influencing the incorporation of past gradients in the current update
            weight_decay = 0.0005,                                         # L2 regularization term
            pretrained   = True,                                           # Enabling use of a pretrained model
            verbose      = False                                           # Shutting off messages in info while training
           )

New https://pypi.org/project/ultralytics/8.3.63 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/josipa/Documents/dataset1/data.yaml, epochs=100, time=None, patience=8, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning /home/josipa/Documents/dataset1/train/labels... 1200 images, 1 backgrounds, 72 corrupt: 100%|██████████| 1200/1200 [00:00<00:00, 3683.24it/s]

train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/0000f2101250b009.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/001b8eac86d79744.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/006df640ce32c5e2.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/017c6dba0c572e61.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/018f64e94e5a3dd8.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/data


train: Scanning /home/josipa/Documents/dataset1/train/labels.cache... 1200 images, 1 backgrounds, 72 corrupt: 100%|██████████| 1200/1200 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/0000f2101250b009.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/001b8eac86d79744.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/006df640ce32c5e2.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/017c6dba0c572e61.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/dataset1/train/images/018f64e94e5a3dd8.jpg: ignoring corrupt image/label: Label class 90 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING ⚠️ /home/josipa/Documents/data

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     11/100         0G      1.172      1.349      1.385         34        640: 100%|██████████| 71/71 [07:48<00:00,  6.59s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.63s/it]

                   all         30          4     0.0778       0.25     0.0348    0.00489



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     12/100         0G      1.134      1.321      1.357         61        640:  96%|█████████▌| 68/71 [07:13<00:18,  6.21s/it]

MAKING PREDICTIONS WITH TRAINED MODEL

In [ ]:
output_dir = './results_predict/'                    # Name of a directory where the results of the prediction will be stored
os.makedirs(output_dir, exist_ok = True)             # Creating a directory

In [ ]:
model.predict(     source = './data/test/images/', # Path to test data
                     save = True,        # Enabling saving of results
                  project = output_dir,  # Name of a directory where the results will be stored
               line_width = 2, 
                     conf = 0.4,         # Confidence threshold for prediction
                     iou  = 0.5,         # IoU threshold for prediction
              show_labels = True,        # Enabling showing labels on images
                  verbose = False,       # Suppressing detailed logs, only critical information will be displayed 
                 save_txt = True,
                save_conf = True,
                imgsz     = 720,         # Defining imgsz
             agnostic_nms = True         # Allowing nms process when making predictions
             )

CALCULATING METRICS ON A VALIDATION DATA

In [ ]:
output_dir_validation = './results_validation/'    # Name of a directory where the results of the validation on a validation data will be stored
os.makedirs(output_dir_validation, exist_ok=True)  # Creating a directory

In [ ]:
# Starting validation process on validation data and storing results in results_val variable
results_val = model.val(data    = "./data/data.yaml",
                        save    = True,
                         iou    = 0.5,
                        conf    = 0.4,
                        project = output_dir_validation)

In [ ]:
# Printing key metrics
print("Mean average precision at IoU=0.5:", round(results_val.box.map50, 4), "\n")
print("Mean average precision at IoU averaged over thresholds 0.5 to 0.95: ", round(results_val.box.map, 4), "\n")    
print("Precision:", results_val.box.f1, "\n")
print("Recall:", results_val.box.r, "\n")
print("AP scores for all classes and all IoU thresholds:", results_val.box.all_ap)

CALCULATING METRICS ON A TEST DATA

In [ ]:
output_dir_test = './results_test/'  # Name of a directory where the results of the validation on a validation data will be stored
os.makedirs(output_dir_test, exist_ok=True)             # Creating a directory

In [ ]:
results_test = model.val(data    = "./data/data.yaml",
                         split   = "test",
                         save    = True,
                          iou    = 0.5,
                         conf    = 0.4,
                         project = output_dir_test)

In [ ]:
# Key metrics
print("Mean average precision at IoU = 0.5:", round(results_test.box.map50, 4), "\n")
print("Mean average precision at IoU averaged over thresholds 0.5 to 0.95: ", round(results_test.box.map, 4), "\n")  
print("Precision:", results_test.box.f1, "\n")
print("Recall:", results_test.box.r, "\n")
print("AP scores for all classes and all IoU thresholds:", results_test.box.all_ap)